In [1]:
from pprint import pprint
from peer_review import *

In [65]:
def evaluate_vancouver(num_assignments, num_reviews, num_truths):
    # generate random groups, assignments, and reviews
    groups = { sub : [sub + x for x in ['1','2','3']] for sub in [ chr(ord('a') + z) for z in range(num_assignments)]}
    assignments = peer_assignment(groups, num_reviews)
    reviews = random_reviews(assignments)
    
    # generate a random ground truth value for all submissions
    truths = {i: random.random() for i in groups}
    truths_visible = truths.copy()
    
    # make a truths_visible dictionary for the algorithm to have access to
    truths_visible = truths.copy()
    _ = {truths_visible.popitem() for i in range(num_assignments - num_truths)}
    
    # run vancouver and omniscient vancouver
    scores, qualities = vancouver(reviews, truths_visible, 1000)
    omni_scores, omni_qualities = vancouver(reviews, truths, 1000)
    
    
    # generate statistics on the data

    '''The bug'''
    
    #this should be the same as the commented code, but isn't
    #sub_score_error = {scores[submission][0] - omni_scores[submission][0] for submission in scores}
    
    sub_score_error = []
    for submission in scores:
        sub_score_error.append(scores[submission][0] - omni_scores[submission][0])
    
    '''/ The Bug '''
          
    sub_var_error = {scores[submission][1] - omni_scores[submission][1] for submission in scores}
    grader_var_error = {qualities[grader] - omni_qualities[grader] for grader in qualities}
    
    # perform a sanity check (shouldn't be necessary, but the interpreter is buggy)
    assert len(sub_score_error) == num_assignments
    assert len(sub_var_error) == num_assignments
    assert len(grader_var_error) == num_assignments * num_reviews
    
    return sub_score_error, sub_var_error, grader_var_error
    
sub_score_error, sub_var_error, grader_var_error = evaluate_vancouver(20, 3, 5)
